# axerunt

In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
from functools import reduce
import logging
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname
    
def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fix_path(get_cwd('axerunt_test.ipynb', 'mutate' +sep))

import numpy as np
import pandas as pd
from dask import delayed

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', 50)

from common_util import MUTATE_DIR, DT_HOURLY_FREQ, DT_CAL_DAILY_FREQ, is_valid, is_type, null_fn, identity_fn, pd_abs, pd_is_empty, ser_range_center_clip
from common_util import load_json, dump_json, pd_slot_shift, arr_nonzero, get_variants, remove_dups_list, list_get_dict, is_empty_df, search_df, benchmark
from data.data_api import DataAPI
from mutate.common import GRAPHS_DIR, HISTORY_DIR, TRANSFORMS_DIR, get_graphs, get_transforms
from mutate.runt import get_rm_keychain, process_transform
from runt_util import RUNT_TYPE_MAPPING, RUNTFormatError, get_ser_fn

DEBUG:matplotlib:$HOME=/home/kev
DEBUG:matplotlib:CONFIGDIR=/home/kev/.config/matplotlib
DEBUG:matplotlib:matplotlib data path: /home/kev/miniconda3/lib/python3.6/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:loaded rc file /home/kev/miniconda3/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc
DEBUG:matplotlib:matplotlib version 3.0.3
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'site', 'os', 'errno', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'sysconfig', '_sysconfigdata_m_linux_x86_64-linux-gnu', 'types', 'functools', '_functools', 'colle

DEBUG:matplotlib:CACHEDIR=/home/kev/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/kev/.cache/matplotlib/fontlist-v300.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


## Init DataAPI

In [2]:
DataAPI.__init__()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
graphs, transforms = get_graphs(), get_transforms()

## #Set Transform

In [3]:
info = transforms['dc']

In [4]:
info

{'meta': {'rec_fmt': 'dc',
  'var_fmt': 'grid',
  'res_freq': 'D',
  'info': 'daily close aggregation'},
 'fn': {'ser_fn': 'sr', 'df_fn': 'gua', 'col_fn': None, 'freq': 'D'},
 'axe': {'rm': ['hrm', 'hrm'],
  'src': [['root', 'root_trmi3_buzz_news'],
   ['root', 'root_trmi3_buzz_social'],
   ['root', 'root_trmi3_nonbuzz_news'],
   ['root', 'root_trmi3_nonbuzz_social']],
  'dst': ['dc', 'dc']},
 'var': {'val': [True], 'flt': [-1]}}

# System Test Transform

In [5]:
res = list(process_transform(info, yield_data=True))

/home/kev/miniconda3/lib/python3.6/site-packages/pyarrow/pandas_compat.py:698: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/home/kev/miniconda3/lib/python3.6/site-packages/pyarrow/pandas_compat.py:725: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/home/kev/miniconda3/lib/python3.6/site-packages/pyarrow/pandas_compat.py:742: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


INFO:root:data: ['dow_jones', 'root', 'root_trmi3_buzz_news', 'join', 'trmi3_etf3']
DEBUG:root:rm_match: [['dow_jones', 'hrm', 'hrm', 'trmi3', 'trmi3']]
DEBUG:root:rm_df.index - src_df.index: DatetimeIndex(['2017-12-16 16:00:00+00:00', '2017-12-16 17:00:00+00:00',
               '2017-12-16 18:00:00+00:00', '2017-12-16 19:00:00+00:00',
               '2017-12-16 20:00:00+00:00', '2017-12-16 21:00:00+00:00',
               '2017-12-17 13:00:00+00:00', '2017-12-17 14:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='id', freq=None)
DEBUG:root:pre-runt:                             etf3_buzz_N
id                                     
1998-01-02 14:00:00+00:00    210.500000
1998-01-02 15:00:00+00:00    210.500000
1998-01-02 16:00:00+00:00    210.500000
1998-01-02 17:00:00+00:00    210.500000
1998-01-02 18:00:00+00:00    730.400000
1998-01-02 19:00:00+00:00    730.400000
1998-01-02 20:00:00+00:00    730.400000
1998-01-02 21:00:00+00:00    730.400000
1998-01-03 13:00:00+00:00    

INFO:root:data: ['dow_jones', 'root', 'root_trmi3_buzz_news', 'join', 'trmi3_mkt3']
DEBUG:root:rm_match: [['dow_jones', 'hrm', 'hrm', 'trmi3', 'trmi3']]
DEBUG:root:rm_df.index - src_df.index: DatetimeIndex(['2003-10-26 17:00:00+00:00', '2017-12-16 16:00:00+00:00',
               '2017-12-16 17:00:00+00:00', '2017-12-16 18:00:00+00:00',
               '2017-12-16 19:00:00+00:00', '2017-12-16 20:00:00+00:00',
               '2017-12-16 21:00:00+00:00', '2017-12-17 13:00:00+00:00',
               '2017-12-17 14:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='id', freq=None)
DEBUG:root:pre-runt:                            mkt3_stockIndexBuzz_N  mkt3_bondBuzz_N  \
id                                                                  
1998-01-02 14:00:00+00:00              62.000000             59.0   
1998-01-02 15:00:00+00:00              62.000000             59.0   
1998-01-02 16:00:00+00:00              62.000000             59.0   
1998-01-02 17:00:00+00:00              6

DEBUG:root:variant: {'val': True, 'flt': -1}
DEBUG:root:subvar: {'val': True, 'flt': -1}
DEBUG:root:mutate_type: dc
DEBUG:root:post-runt:                            mkt3_stockIndexBuzz_N  mkt3_bondBuzz_N  \
id                                                                  
1998-01-02 00:00:00+00:00             398.500000            333.0   
1998-01-03 00:00:00+00:00             162.900000            136.0   
1998-01-04 00:00:00+00:00              34.500000             17.2   
1998-01-05 00:00:00+00:00             301.000000            311.9   
1998-01-06 00:00:00+00:00             416.900000            522.7   
1998-01-07 00:00:00+00:00             872.400000           1215.4   
1998-01-08 00:00:00+00:00            1119.000000           1025.8   
1998-01-09 00:00:00+00:00            1416.300000           1110.0   
1998-01-10 00:00:00+00:00             664.400000            169.0   
1998-01-11 00:00:00+00:00             134.500000            212.2   
1998-01-12 00:00:00+00:00         

INFO:root:data: ['nasdaq_100', 'root', 'root_trmi3_buzz_news', 'join', 'trmi3_etf3']
DEBUG:root:rm_match: [['nasdaq_100', 'hrm', 'hrm', 'trmi3', 'trmi3']]
DEBUG:root:rm_df.index - src_df.index: DatetimeIndex(['2017-12-16 16:00:00+00:00', '2017-12-16 17:00:00+00:00',
               '2017-12-16 18:00:00+00:00', '2017-12-16 19:00:00+00:00',
               '2017-12-16 20:00:00+00:00', '2017-12-16 21:00:00+00:00',
               '2017-12-17 13:00:00+00:00', '2017-12-17 14:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='id', freq=None)
DEBUG:root:pre-runt:                              etf3_buzz_N
id                                      
1998-01-02 14:00:00+00:00      67.000000
1998-01-02 15:00:00+00:00      67.000000
1998-01-02 16:00:00+00:00      67.000000
1998-01-02 17:00:00+00:00      67.000000
1998-01-02 18:00:00+00:00     412.500000
1998-01-02 19:00:00+00:00     412.500000
1998-01-02 20:00:00+00:00     412.500000
1998-01-02 21:00:00+00:00     412.500000
1998-01-03 13:00:

...

INFO:root:data: ['nasdaq_100', 'root', 'root_trmi3_nonbuzz_news', 'join', 'trmi3_mkt3_stock']
DEBUG:root:rm_match: [['nasdaq_100', 'hrm', 'hrm', 'trmi3', 'trmi3']]
DEBUG:root:rm_df.index - src_df.index: DatetimeIndex(['2001-10-28 17:00:00+00:00', '2002-10-27 17:00:00+00:00',
               '2003-07-20 17:00:00+00:00', '2003-07-20 18:00:00+00:00',
               '2003-07-20 19:00:00+00:00', '2003-07-20 20:00:00+00:00',
               '2003-10-26 17:00:00+00:00', '2004-12-25 19:00:00+00:00',
               '2004-12-25 20:00:00+00:00', '2004-12-25 21:00:00+00:00',
               '2004-12-26 13:00:00+00:00', '2004-12-26 14:00:00+00:00',
               '2004-12-26 15:00:00+00:00', '2017-12-16 16:00:00+00:00',
               '2017-12-16 17:00:00+00:00', '2017-12-16 18:00:00+00:00',
               '2017-12-16 19:00:00+00:00', '2017-12-16 20:00:00+00:00',
               '2017-12-16 21:00:00+00:00', '2017-12-17 13:00:00+00:00',
               '2017-12-17 14:00:00+00:00'],
              dtype='d

DEBUG:root:variant: {'val': True, 'flt': -1}
DEBUG:root:subvar: {'val': True, 'flt': -1}
DEBUG:root:mutate_type: dc
DEBUG:root:post-runt:                            mkt3_stockIndexSentiment_N  \
id                                                      
1998-01-02 00:00:00+00:00                    0.109159   
1998-01-03 00:00:00+00:00                    0.089012   
1998-01-04 00:00:00+00:00                    0.304348   
1998-01-05 00:00:00+00:00                   -0.064784   
1998-01-06 00:00:00+00:00                    0.004797   
1998-01-07 00:00:00+00:00                   -0.282554   
1998-01-08 00:00:00+00:00                   -0.336908   
1998-01-09 00:00:00+00:00                   -0.270070   
1998-01-10 00:00:00+00:00                   -0.328116   
1998-01-11 00:00:00+00:00                   -0.263941   
1998-01-12 00:00:00+00:00                   -0.189530   
1998-01-13 00:00:00+00:00                    0.022908   
1998-01-14 00:00:00+00:00                    0.038885   
1998-01

INFO:root:data: ['russell_2000', 'root', 'root_trmi3_nonbuzz_news', 'join', 'trmi3_etf3_forecast']
DEBUG:root:rm_match: [['russell_2000', 'hrm', 'hrm', 'trmi3', 'trmi3']]
DEBUG:root:rm_df.index - src_df.index: DatetimeIndex(['2017-12-16 16:00:00+00:00', '2017-12-16 17:00:00+00:00',
               '2017-12-16 18:00:00+00:00', '2017-12-16 19:00:00+00:00',
               '2017-12-16 20:00:00+00:00', '2017-12-16 21:00:00+00:00',
               '2017-12-17 13:00:00+00:00', '2017-12-17 14:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='id', freq=None)
DEBUG:root:pre-runt:                            etf3_timeUrgency_N  etf3_uncertainty_N  \
id                                                                  
1998-01-02 14:00:00+00:00           -0.020619            0.010309   
1998-01-02 15:00:00+00:00           -0.020619            0.010309   
1998-01-02 16:00:00+00:00           -0.020619            0.010309   
1998-01-02 17:00:00+00:00           -0.020619            0.010309 

DEBUG:root:variant: {'val': True, 'flt': -1}
DEBUG:root:subvar: {'val': True, 'flt': -1}


KeyboardInterrupt: 

# Debug Transform - Init Transform Settings

In [7]:
meta, fn, axe, var = info['meta'], info['fn'], info['axe'], info['var']

if (meta['var_fmt']=='list' and any([s in meta['rec_fmt'] for s in ('{', '}')])):
    error_msg = 'cannot mix var_fmt==\'list\' with a parameter-inputed rec_fmt'
    logging.error(error_msg)
    raise RUNTFormatError(error_msg)

### Loading transform, apply, and frequency settings

In [8]:
rtype_fn = RUNT_TYPE_MAPPING.get(fn['df_fn'])
ser_fn_str, col_fn_str = fn['ser_fn'], fn['col_fn']
freq, res_freq = fn['freq'], meta['res_freq']

### Variants and type format string

In [9]:
variants = get_variants(var, meta['var_fmt'])
print(variants)
type_str = meta['rec_fmt']

[({'val': True, 'flt': 0}, {'val': True, 'flt': 'h'}, {'val': True, 'flt': 'l'}, {'val': True, 'flt': -1}, {'stat_type': 'avg'})]


### Prep input data and row masks

In [10]:
srcs = axe['src'] if (is_type(axe['src'][0], list)) else [axe['src']]
rm_src = axe['rm']
if (is_valid(rm_src)):
    rm_rcs, rm_dfs = DataAPI.axe_load(rm_src, lazy=False)

## #Debug Transform - Load Input Data

In [11]:
src = srcs[0]
src_rcs, src_dfs = DataAPI.axe_load(src, lazy=False)
dfkeys = list(src_dfs.keys())

In [12]:
keychain = dfkeys[0]
src_df = src_dfs[keychain]

### Row Mask Input Data

In [13]:
if (is_valid(rm_src)):
    rm_keychain = get_rm_keychain(keychain, rm_dfs.keys())
    rm_df = rm_dfs[rm_keychain].dropna()
    rm_src_diff = rm_df.index.difference(src_df.index)
    if (len(rm_src_diff)>0):
        logging.debug('rm_df.index - src_df.index: {}'.format(str(rm_src_diff)))
        src_df = src_df.loc[src_df.index & rm_df.index, :].dropna(axis=0, how='all')
    else:
        src_df = src_df.loc[rm_df.index, :].dropna(axis=0, how='all')

DEBUG:root:rm_match: [['dow_jones', 'hrm', 'hrm', 'pba', 'pba']]


In [14]:
src_df.head(10)

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,
1998-01-02 14:00:00+00:00,7910.24,7952.59,7910.24,7923.19,7933.6658
1998-01-02 15:00:00+00:00,7923.19,7923.19,7880.85,7917.22,7904.5360
1998-01-02 16:00:00+00:00,7917.22,7931.66,7914.48,7916.22,7922.5204
1998-01-02 17:00:00+00:00,7915.72,7940.38,7915.72,7934.90,7930.3804
1998-01-02 18:00:00+00:00,7934.65,7935.40,7916.22,7916.22,7927.0547
1998-01-02 19:00:00+00:00,7916.22,7920.20,7896.29,7908.99,7909.1580
1998-01-02 20:00:00+00:00,7910.24,7957.07,7910.24,7955.33,7933.0036
1998-01-02 21:00:00+00:00,7956.08,7965.04,7955.33,7965.04,7959.8912
1998-01-05 14:00:00+00:00,7967.04,7989.96,7952.59,7963.80,7970.0015


### #Set Variant

In [15]:
variant = variants[0]
print(variant)

({'val': True, 'flt': 0}, {'val': True, 'flt': 'h'}, {'val': True, 'flt': 'l'}, {'val': True, 'flt': -1}, {'stat_type': 'avg'})


## Debug Transform - Load Functions and Variants

In [16]:
fns = get_ser_fn(fn['ser_fn'], variant)
print(fns)

DEBUG:root:subvar: {'val': True, 'flt': 0}
DEBUG:root:subvar: {'val': True, 'flt': 'h'}
DEBUG:root:subvar: {'val': True, 'flt': 'l'}
DEBUG:root:subvar: {'val': True, 'flt': -1}
DEBUG:root:subvar: {'stat_type': 'avg'}
[<function single_row.<locals>.fn at 0x7f0d754cd510>, <function single_row.<locals>.fn at 0x7f0d754cd950>, <function single_row.<locals>.fn at 0x7f0d7535a6a8>, <function single_row.<locals>.fn at 0x7f0d754e5730>, <function Series.mean at 0x7f0e03839400>]


## Debug Transform - Isolated Transform Testing

In [17]:
logging.info(fn['df_fn'])
logging.info(freq)
if (fn['df_fn'] == 'rut'):
    pass
elif (fn['df_fn'] == 'rbt'):
    pass
elif (fn['df_fn'] == 'gut'):
    pass
elif (fn['df_fn'] == 'gua'):
    for i, col in enumerate(src_df.columns):
        ser = src_df.iloc[:, i]
        logging.info('{c}: {s}'.format(c=col, s=ser))
        print(ser.groupby(pd.Grouper(freq=freq)).agg(fns[-pd.Series(data=[], index=[]).idxmax()]))

INFO:root:gua
INFO:root:H
INFO:root:pba_open: id
1998-01-02 14:00:00+00:00     7910.24
1998-01-02 15:00:00+00:00     7923.19
1998-01-02 16:00:00+00:00     7917.22
1998-01-02 17:00:00+00:00     7915.72
1998-01-02 18:00:00+00:00     7934.65
1998-01-02 19:00:00+00:00     7916.22
1998-01-02 20:00:00+00:00     7910.24
1998-01-02 21:00:00+00:00     7956.08
1998-01-05 14:00:00+00:00     7967.04
1998-01-05 15:00:00+00:00     7965.04
1998-01-05 16:00:00+00:00     7997.43
1998-01-05 17:00:00+00:00     8030.06
1998-01-05 18:00:00+00:00     7996.43
1998-01-05 19:00:00+00:00     7961.81
1998-01-05 20:00:00+00:00     7962.05
1998-01-05 21:00:00+00:00     7971.77
1998-01-06 14:00:00+00:00     7978.25
1998-01-06 15:00:00+00:00     7932.16
1998-01-06 16:00:00+00:00     7930.17
1998-01-06 17:00:00+00:00     7900.03
1998-01-06 18:00:00+00:00     7908.00
1998-01-06 19:00:00+00:00     7926.93
1998-01-06 20:00:00+00:00     7910.24
1998-01-06 21:00:00+00:00     7907.75
1998-01-07 14:00:00+00:00     7904.76
1

ValueError: attempt to get argmax of an empty sequence